[Open PDF](problem-schedulingDay.pdf)

Requrido:

1. función:
Conseguir un listado de días disponibles considerando:

* Días Feriados y fines de semana
* Días No trabajados ( lunes - viernes )
* Días Colapsados ( lunes - viernes )

2. función:
Conseguir fecha de notificación.
Fecha de notificación = fecha del retiro - días de peparación


* Días requeridos para preparar pedidos

In [68]:
import requests
from datetime import datetime, timedelta

In [69]:
api_url = "https://date.nager.at/api/v3/publicholidays/2024/CL"
r = requests.get(api_url)
data = r.json()
holidays = {datetime.strptime(entry['date'], '%Y-%m-%d') for entry in data}
# holidays

In [70]:
collapsedDaysStr = ["2024-05-23","2024-05-24","2024-05-31"]
collapsedDays = {datetime.strptime(element, '%Y-%m-%d') for element in collapsedDaysStr}
# collapsedDays

In [81]:
def availableDaysFromToday(daysFromToday):
  today = datetime.today()
  listDays = [today + timedelta(days=i) for i in range(daysFromToday)]
  return listDays

In [72]:
def filterDays(listTotalDays, setToFilter, nonWorkingdays):
  filteredLists = [ element for element in listTotalDays if element not in setToFilter and element not in nonWorkingdays]
  return filteredLists

In [80]:
API_URL = "https://date.nager.at/api/v3/publicholidays/2024/CL"

def listAvailableDays(collapsedDays = ["2024-05-23","2024-05-24","2024-05-31"], nonWorkingdays = [0, 1, 5, 6], daysFromToday = 30):
  listTotalDays =  availableDaysFromToday(daysFromToday)
  r = requests.get(API_URL)
  data = r.json()
  holidays = {datetime.strptime(entry['date'], '%Y-%m-%d') for entry in data}
  collapsedDays = {datetime.strptime(element, '%Y-%m-%d') for element in collapsedDaysStr}
  setToFilter = holidays | collapsedDays
  finalList = filterDays(listTotalDays, setToFilter, nonWorkingdays)
  # finalListStr = [element.strftime('%Y-%m-%d') for element in finalList]
  return finalList

In [79]:
listDays = listAvailableDays()
# listDays

In [77]:
def notificationDate(availableDays = listDays, preparationDays = 20):
  return availableDays[preparationDays].strftime('%Y-%m-%d')


In [78]:
notificationDate()

'2024-06-15'

Now with Class

Available Days: ['2024-05-22', '2024-05-23', '2024-05-24', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-05', '2024-06-06', '2024-06-07', '2024-06-12', '2024-06-13', '2024-06-14', '2024-06-19', '2024-06-20']
Notification Day: 2024-05-24


# Class answer

In [113]:
import requests
from datetime import datetime, timedelta

class schedulingDay:
    def __init__(self, api_url, collapsedDaysStr, nonWorkingDays, daysFromToday, preparationDays):
        self.api_url = api_url
        self.availableDays = []
        self.availableTotalDays = []
        self.nonWorkingDays = nonWorkingDays
        self.daysFromToday = daysFromToday
        self.preparationDays = preparationDays
        self.holidays = set()
        self.collapsedDays = {datetime.strptime(element, '%Y-%m-%d').date() for element in collapsedDaysStr}
    
    def fetch_data(self):
        response = requests.get(self.api_url)
        response.raise_for_status()
        self.holidays = {datetime.strptime(entry['date'], '%Y-%m-%d').date() for entry in response.json()}
    
    def availableDaysFromToday(self):
        today = datetime.today().date()
        self.availableTotalDays = [today + timedelta(days=i) for i in range(self.daysFromToday)]
    
    def filterDays(self, setToFilter):
        self.availableDays = [element for element in self.availableTotalDays 
                              if element not in setToFilter and element.weekday() not in self.nonWorkingDays]

    def notificationDate(self):
        now = datetime.now() # current time
        one_pm = now.replace(hour=13, minute=0, second=0, microsecond=0) #1 pm
        if now > one_pm:
            self.preparationDays +=1
        if len(self.availableDays) > self.preparationDays:
            return self.availableDays[self.preparationDays]
        return None
    
    def listAvailableDays(self):
        self.fetch_data()
        self.availableDaysFromToday()
        setToFilter = self.holidays | self.collapsedDays
        self.filterDays(setToFilter)
        notificationDay = self.notificationDate()
        return self.availableDays, notificationDay

# Usage
api_url = "https://date.nager.at/api/v3/publicholidays/2024/CL"
collapsedDays = ["2024-05-23", "2024-05-24", "2024-05-31"]
nonWorkingDays = [0, 1, 5, 6]  # Monday, Tuesday, Saturday, Sunday
daysFromToday = 30
preparationDays = 2

# Create an instance of schedulingDay
schedulingDay_api = schedulingDay(api_url, collapsedDays, nonWorkingDays, daysFromToday, preparationDays)

# Get the available days and the notification day
available_days, notification_day = schedulingDay_api.listAvailableDays()

# Print the results
print(f"Available Days: {[day.strftime('%Y-%m-%d') for day in available_days]}")
print(f"Notification Day: {notification_day.strftime('%Y-%m-%d')}")

Available Days: ['2024-05-22', '2024-05-29', '2024-05-30', '2024-06-05', '2024-06-06', '2024-06-12', '2024-06-13', '2024-06-14', '2024-06-19']
Notification Day: 2024-06-05
